# API Dataset

In [2]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import style
import urllib.request, urllib.parse
from urllib.error import HTTPError, URLError
from urllib.request import urlretrieve as retrieve
import json
import requests
from bs4 import BeautifulSoup
import pprint
import csv
import netCDF4
from pandasql import sqldf
import sqlite3
import datetime

C:\Users\datre\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
url = "https://cmr.earthdata.nasa.gov/opensearch/granules/descriptor_document.xml?clientId=eed-edsc-prod-serverless-client&shortName=CMS_Monthly_CO2_Gulf_of_Mexico_1668&versionId=1&dataCenter=ORNL_DAAC"
r = requests.get(url)
print("CONTENT", r.content)

CONTENT b'<?xml version="1.0"?>\n<os:OpenSearchDescription\n\txmlns:os="http://a9.com/-/spec/opensearch/1.1/"\n\txmlns:echo="https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html#atom"\n\txmlns:geo="http://a9.com/-/opensearch/extensions/geo/1.0/"\n\txmlns:time="http://a9.com/-/opensearch/extensions/time/1.0/"\n\txmlns:esipdiscovery="http://commons.esipfed.org/ns/discovery/1.2/" esipdiscovery:version="1.2"\n\txmlns:params="http://a9.com/-/spec/opensearch/extensions/parameters/1.0/"\n\txmlns:referrer="http://www.opensearch.org/Specifications/OpenSearch/Extensions/Referrer/1.0"\n\txmlns:eo="http://a9.com/-/opensearch/extensions/eo/1.0/"\n\txmlns:atom="http://www.w3.org/2005/Atom">\n\t<os:ShortName>CMR Granules</os:ShortName>\n\t<os:Description>NASA CMR Granule search using geo, time and parameter extensions</os:Description>\n\t<os:Tags>CMR NASA CWIC CEOS-OS-BP-V1.1/L3 ESIP OGC granule pageOffset=1 indexOffset=0</os:Tags>\n\t<os:Contact>echodev@echo.nasa.gov</os:Contact>\n\t<os:U

need: template="https://cmr.earthdata.nasa.gov/opensearch/granules.html?datasetId={echo:datasetId?}&amp;shortName=CMS_Monthly_CO2_Gulf_of_Mexico_1668&amp;versionId=1&amp;dataCenter=ORNL_DAAC&amp;boundingBox={geo:box?}&amp;geometry={geo:geometry?}&amp;placeName={geo:name?}&amp;startTime={time:start?}&amp;endTime={time:end?}&amp;cursor={os:startPage?}&amp;numberOfResults={os:count?}&amp;offset={os:startIndex?}&amp;uid={geo:uid?}&amp;parentIdentifier={eo:parentIdentifier?}&amp;clientId=eed-edsc-prod-serverless-client"

In [8]:
path = r"C:\\Users\\datre\\OneDrive\\Documents\\Graduate School\\Summer '20\\pco2_co2_flux.nc"
nc = netCDF4.Dataset(path)
nc

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    Conventions: CF-1.6
    title: Monthly Satellite-Estimated pCO2 and air-sea flux of CO2 for Northern Gulf of Mexico, 2006-2010
    institution: Oak Ridge National Laboratory (ORNL) Distributed Active Archive Center (DAAC)
    references: Lohrenz, S.E., Cai, W.J., Chakraborty, S., Huang, W.J., Guo, X., He, R., Xue, Z., Fennel, K., Howden, S. and Tian, H. 2018. Satellite estimation of coastal pCO2 and air-sea flux of carbon dioxide in the northern Gulf of Mexico. Remote Sensing of Environment, 207:71-83. doi.org/10.1016/j.rse.2017.12.039
    source: Version 1, funding from NASA's Carbon Monitoring System
    dimensions(sizes): time(60), nv(2), lon(960), lat(672)
    variables(dimensions): float64 time(time), float64 time_bnds(time, nv), float64 CO2_flux(time, lon, lat), float64 PCO2(time, lon, lat), float64 lat(lat), float64 lon(lon), float64 crs()
    groups: 

In [9]:
for i in nc.variables:
    print(i, nc.variables[i].units, nc.variables[i].shape)

time days since 2006-01-01 (60,)
time_bnds days since 2006-01-01 (60, 2)
CO2_flux mmol m-2 s-1 (60, 960, 672)
PCO2 Pa (60, 960, 672)
lat degrees_north (672,)
lon degrees_east (960,)


AttributeError: NetCDF: Attribute not found

In [10]:
date = nc.variables["time"][:]
date_bnd = nc.variables["time_bnds"][:]
CO2_flux = nc.variables["CO2_flux"][:]
pCO2 = nc.variables["PCO2"][:]
lat = nc.variables["lat"][:]
lon = nc.variables["lon"][:]

In [11]:
date

masked_array(data=[  15.5,   45. ,   74.5,  105. ,  135.5,  166. ,  196.5,
                    227.5,  258. ,  288.5,  319. ,  349.5,  380.5,  410. ,
                    439.5,  470. ,  500.5,  531. ,  561.5,  592.5,  623. ,
                    653.5,  684. ,  714.5,  745.5,  775.5,  805.5,  836. ,
                    866.5,  897. ,  927.5,  958.5,  989. , 1019.5, 1050. ,
                   1080.5, 1111.5, 1141. , 1170.5, 1201. , 1231.5, 1262. ,
                   1292.5, 1323.5, 1354. , 1384.5, 1415. , 1445.5, 1476.5,
                   1506. , 1535.5, 1566. , 1596.5, 1627. , 1657.5, 1688.5,
                   1719. , 1749.5, 1780. , 1810.5],
             mask=False,
       fill_value=1e+20)

In [12]:
data = {date, CO2_flux, pCO2, lat, lon}
api = pd.DataFrame(data, columns = ("Date", "CO2_flux", "pCO2", "lat", "lon"))

TypeError: unhashable type: 'MaskedArray'